In [1]:
# set seed for reproducibility purposes.
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [2]:
import tensorflow as tf
import numpy as np
import os
from time import strftime

from PIL import Image

In [3]:
# CONSTANTS
X_TRAIN_PATH = 'MNIST_Dataset/digit_xtrain.csv'
X_TEST_PATH = 'MNIST_Dataset/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST_Dataset/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST_Dataset/digit_ytest.csv'

SINGLE_IMG_PATH = 'MNIST_Dataset/test_img.png'

EXPORT_PATH = 'SavedModel_digit-recog'

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
NUM_CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * NUM_CHANNELS
NUM_CLASSES = 10 # digits from 0-9
VALIDATION_SIZE = 10000

LOGGING_PATH = 'tensorboard_mnist_logs/'

## *Load Data*

In [4]:
%%time

y_train_data = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
print('Y Train shape: ', y_train_data.shape)

y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
print('Y Test shape: ', y_test.shape)

Y Train shape:  (60000,)
Y Test shape:  (10000,)
CPU times: user 216 ms, sys: 667 µs, total: 216 ms
Wall time: 220 ms


In [5]:
%%time

x_train_data = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
print('X Train shape: ', x_train_data.shape)

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
print('X Test shape: ', x_test.shape)

X Train shape:  (60000, 784)
X Test shape:  (10000, 784)
CPU times: user 29.2 s, sys: 1.57 s, total: 30.8 s
Wall time: 31 s


---

## *Data preprocessing*

In [6]:
# re-scale training data from 0-255 to 0-1
x_train_data, x_test = x_train_data/255.0, x_test/255.0

# convert target values (labels) to one-hot-encoding
y_train_data = np.eye(NUM_CLASSES)[y_train_data]
print('Y Train shape: ', y_train_data.shape)

y_test = np.eye(NUM_CLASSES)[y_test]
print('Y Test shape: ', y_test.shape)

Y Train shape:  (60000, 10)
Y Test shape:  (10000, 10)


### *Create validation dataset from training data*

In [7]:
x_val = x_train_data[:VALIDATION_SIZE]
y_val = y_train_data[:VALIDATION_SIZE]

x_train = x_train_data[VALIDATION_SIZE:]
y_train = y_train_data[VALIDATION_SIZE:]

print('Validation shape: ', x_val.shape, y_val.shape)
print('New training shape: ', x_train.shape, y_train.shape)


Validation shape:  (10000, 784) (10000, 10)
New training shape:  (50000, 784) (50000, 10)


---

## *Setup tensorflow graph*

In [8]:
# https://www.tensorflow.org/api_docs/python/tf/compat/v1/placeholder

# from docs: "... This API is strongly discouraged for use with eager execution and tf.function."
tf.compat.v1.disable_eager_execution()

# features placeholder
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')

#labels placeholder
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NUM_CLASSES], name='Labels')

### *Neural network definition*

In [9]:
# hyperparameters
num_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512 # layer 1
n_hidden2 = 64 # layer 2

model_name = f'{n_hidden1}-DO-{n_hidden2}_LR-{learning_rate}_E{num_epochs}'

### *Generic layer setup*

In [10]:
def layer_setup(input, weigth_dim, bias_dim, name):
    with tf.name_scope(name):
        # setup weights and biases
        initial_w = tf.random.truncated_normal(shape=weigth_dim, stddev=0.1)
        w = tf.Variable(initial_value=initial_w, name='W')
        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')
        
        #setup inputs and outputs
        layer_in  = tf.matmul(input, w) + b
        
        if name=='out':
            # output layer use softmax
            layer_out = tf.nn.softmax(layer_in)
        else:
            # inner layer use relu
            layer_out = tf.nn.relu(layer_in)
            
        # add histogram summaries for w and b
        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)
        
        return layer_out

#### *Setup 1st layer*

In [11]:
# with tf.name_scope('hidden_1'):
#     # set initial random weights for 1st layer
#     initial_w1 = tf.random.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1)
#     # create tf variable for weights
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     # set initial bias for 1st layer
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     # https://www.tensorflow.org/api_docs/python/tf/linalg/matmul

#     # set 1st layer inputs (X which is the raw inputs multiplied by the weights (matrix mult) + bias)
#     # this result is what feeds the activation function
#     layer1_in = tf.matmul(X, w1) + b1

#     # set 1st layer outputs using rectified linear (activation function)
#     layer1_out = tf.nn.relu(layer1_in)

h1_output = layer_setup(input=X, weigth_dim=[TOTAL_INPUTS, n_hidden1],
                        bias_dim=[n_hidden1], name='hidden_1')

# add dropout layer
dropout_layer_output = tf.compat.v1.nn.dropout(h1_output, rate=0.2, name='dropout_layer')

##### *Setup 2nd layer*

In [12]:
# with tf.name_scope('hidden_2'):
#     # setup weights and biases
#     initial_w2 = tf.random.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')
#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     # setup inputs and outputs
#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

h2_output = layer_setup(input=dropout_layer_output, weigth_dim=[n_hidden1, n_hidden2],
                        bias_dim=[n_hidden2], name='hidden_2')

#### *Setup output layer*

In [13]:
# with tf.name_scope('output_layer'):
#     # setup weights and biases
#     initial_w3 = tf.random.truncated_normal(shape=[n_hidden2, NUM_CLASSES], stddev=0.1)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')
#     initial_b3 = tf.constant(value=0.0, shape=[NUM_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     # setup inputs and outputs
#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     final_output = tf.nn.softmax(layer3_in) # softmax is used as activation function because it's the output layer

final_output = layer_setup(input=h2_output, weigth_dim=[n_hidden2, NUM_CLASSES],
                     bias_dim=[NUM_CLASSES], name='out')

### *Tensorboard Setup*

In [14]:
folder_name = f'{model_name}_{strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Directory created.')

Directory created.


### Loss function

In [15]:
with tf.name_scope('loss_calc'):
    # https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits

    # reduce_mean is used to get the average of the losses as the training
    # will happen in batches instead of the whole dataset.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=final_output))

### Optimizer

In [16]:
with tf.name_scope('optimizer'):
    # https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

    # https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/AdamOptimizer#minimize
    train_step = optimizer.minimize(loss)

### Accuracy

In [17]:
with tf.name_scope('accuracy_calc'):
    # argmax of the final_output row has the highest probability from indexes 0-9 (softmax output)
    # argmax of the Y row has 1 for the true label and 0 for the rest from indexes 0-9 (one-hot-encoding)
    model_prediction = tf.argmax(final_output, axis=1, name='prediction')
    correct_prediction = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    # calculate accuracy
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
with tf.name_scope('performance'):
    # (tensorboard)
    # add summary for the accuracy
    tf.compat.v1.summary.scalar('accuracy', acc)
    # add summary for the loss
    tf.compat.v1.summary.scalar('loss', loss)

### Display input images in Tensorboard

In [19]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=5)

---

## *Start and run session*

In [20]:
# "A Session object encapsulates the environment in which 
# Operation objects are executed, and Tensor objects are evaluated."
session = tf.compat.v1.Session()

2021-12-21 21:51:44.507360: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-21 21:51:44.507428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (edlcorrea-X555LF): /proc/driver/nvidia/version does not exist
2021-12-21 21:51:44.509696: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### *[Tensorboard] Setup FileWriter and merge summaries*

In [21]:
# setup FileWriter and merge summaries (tensorboard)
merged_summary = tf.compat.v1.summary.merge_all()

# define train writer
train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(session.graph)

# define validation writer
validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

In [22]:
# initialize the variables
init = tf.compat.v1.global_variables_initializer()
session.run(init)

### *Batching the data*

In [23]:
batch_size = 1000
num_examples = y_train.shape[0]
num_iterations = int(num_examples/batch_size)

index_in_epoch = 0

In [24]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        # reset
        start=0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Write the training loop

In [25]:
print('Training started.')
for epoch in range(num_epochs):
    
    # TRAINING DATASET
    for i in range(num_iterations):
        batch_x, batch_y = next_batch(batch_size=batch_size, data=x_train, labels=y_train)
        
        # create feed dictionary
        feed_dictionary = {X: batch_x, Y: batch_y}        
        # do the calculations for training
        session.run(train_step, feed_dict=feed_dictionary)
    
    # fetch the accuracy and summary from session
    train_summary, batch_accuracy = session.run(fetches=[merged_summary, acc], feed_dict=feed_dictionary)
    train_writer.add_summary(train_summary, epoch)
        
    print(f'Epoch {epoch} \t| Training accuracy = {batch_accuracy}')
    
    # VALIDATION DATASET
    val_summary = session.run(fetches=merged_summary, feed_dict={X: x_val, Y: y_val})
    validation_writer.add_summary(val_summary, epoch)
    
print('Training complete.')

Training started.
Epoch 0 	| Training accuracy = 0.8560000061988831
Epoch 1 	| Training accuracy = 0.9369999766349792
Epoch 2 	| Training accuracy = 0.9679999947547913
Epoch 3 	| Training accuracy = 0.972000002861023
Epoch 4 	| Training accuracy = 0.9729999899864197
Epoch 5 	| Training accuracy = 0.9760000109672546
Epoch 6 	| Training accuracy = 0.9800000190734863
Epoch 7 	| Training accuracy = 0.9769999980926514
Epoch 8 	| Training accuracy = 0.9850000143051147
Epoch 9 	| Training accuracy = 0.9860000014305115
Epoch 10 	| Training accuracy = 0.9860000014305115
Epoch 11 	| Training accuracy = 0.9900000095367432
Epoch 12 	| Training accuracy = 0.9879999756813049
Epoch 13 	| Training accuracy = 0.9890000224113464
Epoch 14 	| Training accuracy = 0.9890000224113464
Epoch 15 	| Training accuracy = 0.9909999966621399
Epoch 16 	| Training accuracy = 0.9890000224113464
Epoch 17 	| Training accuracy = 0.9909999966621399
Epoch 18 	| Training accuracy = 0.9890000224113464
Epoch 19 	| Training acc

---
## *Save the model (for future serving)*

In [26]:
# # https://www.tensorflow.org/api_docs/python/tf/compat/v1/saved_model/simple_save
outputs = { 'accuracy_calc/prediction': model_prediction }
inputs = { 'X': X }

tf.compat.v1.saved_model.simple_save(session=session, export_dir=EXPORT_PATH,
                                     inputs=inputs, outputs=outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel_digit-recog/saved_model.pb


---

## *Test a single prediction*

In [27]:
sample_img = Image.open(SINGLE_IMG_PATH)
sample_img

In [28]:
# treat image before supplying it to the model
# to black and white
bw_img = sample_img.convert(mode='L')

# invert colors (dataset has white color with black background)
img_array = np.invert(bw_img)
print(img_array.shape)

# flatten values
flat_img = img_array.ravel()
print(flat_img.shape)

(28, 28)
(784,)


### *Predict for the '2' image*

In [29]:
prediction = session.run(fetches=tf.argmax(final_output, axis=1), feed_dict={ X:[flat_img] })
print(f'Prediction result for the image of number two: {prediction}')

Prediction result for the image of number two: [2]


# *Testing and Evaluation* 

In [30]:
# run the test dataset
test_accuracy = session.run(fetches=acc, feed_dict={X:x_test, Y:y_test})

print(f'Accuracy achieved on test dataset: {test_accuracy:0.2%}')

Accuracy achieved on test dataset: 97.76%


---
## *Final accuracy on test dataset: 97.85%*
---

## *Reset for next run*

In [31]:
train_writer.close()
validation_writer.close()
session.close()
tf.compat.v1.reset_default_graph()

---